In [1]:
!pip install sentence-transformers textstat nltk textblob vaderSentiment langdetect transformers language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=2bea7be01a1934e184c0625fc7ba7549152e7a22cfaeb8efd718ba5744bd92b4
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
!pip install git+https://github.com/PrithivirajDamodaran/Gramformer.git

  Cloning https://github.com/PrithivirajDamodaran/Gramformer.git to /tmp/pip-req-build-zqf4s66n
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Gramformer.git /tmp/pip-req-build-zqf4s66n
  Resolved https://github.com/PrithivirajDamodaran/Gramformer.git to commit 23425cd2e98a919384cab6156af8adf1c9d0639a
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.3/499.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.7 MB/s eta 0:00:00
  Created wheel for gramformer: filename=gramformer-1.0-py3-none-any.whl size=4461 sha256=1bb18eb02cee184848519e256d7cbae2f4ea51812bd7feb2645d129d0450c244
  Stored in directory: /tmp/pip-ephem-wheel-cache-pa4jhlbo/wheels/76/44/15/e79b5dc4f5c897b2054e6a8e357f6de157b9554f072a2e56ea
Successfully built gramformer


In [4]:
import nltk
nltk.download('punkt_tab')
import language_tool_python
import re
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import textstat
from nltk.tokenize import sent_tokenize
from langdetect import detect
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gramformer import Gramformer
import re

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [16]:
# @title Presentation Assessment Pipeline
# Initialize tools and models
spell_checker=language_tool_python.LanguageTool('en-US') # Spell-Checker
gf = Gramformer(models=1)  # Set models=1 for grammar correction
relevancy_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # Relevancy Model
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Semantic similarity
sentiment_analyzer = SentimentIntensityAnalyzer()  # Sentiment analysis
politeness_analyzer= SentimentIntensityAnalyzer()  # Politeness Analyzer
coherency_analyzer= SentenceTransformer('all-MiniLM-L6-v2') # Coherency Analyzer


def presentation_skill_ass(content,topic):
    # Assessment Criteria
    def evaluate_spell_with_textblob(content):
        text = spell_checker.check(content)
        corrected_content = language_tool_python.utils.correct(content,text)
        lines=content.strip().split("\n")
        incorrect=0
        for line in lines:
            if line !="":
              prev_words=line.split(" ")
              incorrect_line = spell_checker.check(line)
              correct_line=language_tool_python.utils.correct(line,incorrect_line)
              new_words=correct_line.split(" ")
              for prev_word,new_word in zip(prev_words,new_words):
                if prev_word != new_word:
                  incorrect+=1
        return incorrect,corrected_content


    def grammatical_error(content):
        #content processing
        sp_lines=content.strip().split("\n")
        lines=[]

        for line in sp_lines:
            lis=line.split(".")
            for l in lis:
              if l!="":
                lines.append(l.strip())

        incorrect=0
        errors={}

        for line in lines:
          if line!="":
              line=re.sub(r"[^\w\s\']", "", line)
              prev_words=line.split(" ")

              # Correct grammar
              corrected_line = gf.correct(line)
              corrected_line_without_pun=next(iter(corrected_line))
              corrected_line_without_pun=re.sub(r"[^\w\s\']","",corrected_line_without_pun)
              new_words=corrected_line_without_pun.split()


              for prev_word,new_word in zip(prev_words,new_words):
                if prev_word != new_word:
                  errors[line]=corrected_line_without_pun
                  incorrect+=1
                  break

        return incorrect,round((10-((incorrect*10)/len(lines))),0) ,errors

    def evaluate_relevance(content, topic):
        topic_embedding = relevancy_model.encode(topic, convert_to_tensor=True)
        speech_embedding = relevancy_model.encode(content, convert_to_tensor=True)

        # Compute similarity
        similarity_score = util.cos_sim(topic_embedding, speech_embedding).item()

        if similarity_score < 0.45:
            similarity_tag = "Irrelevant"
        elif similarity_score < 0.5:
            similarity_tag = "Relevant"
        else:
            similarity_tag = "Highly Relevant"

        return round(similarity_score, 2), similarity_tag

    def evaluate_sentiment(content):
        sentences = sent_tokenize(content)
        scores = [sentiment_analyzer.polarity_scores(sent)['compound'] for sent in sentences]
        avg_sentiment = sum(scores) / len(scores) if scores else 0

        #tagging for sentiment
        if avg_sentiment>0:
            sentiment_tag="Positive"
        elif avg_sentiment<0:
            sentiment_tag="Negative"
        else:
            sentiment_tag="Neutral"
        return round(avg_sentiment,2),sentiment_tag

    def evaluate_readability(content):
        flesch_score = textstat.flesch_reading_ease(content)

        #tagging for readability
        if flesch_score>60:
            flesch_tag="Easy to Read"
        elif 30<flesch_score<60:
            flesch_tag="Moderate Readability"
        else:
            flesch_tag="Difficult to Read"

        return round(flesch_score,2),flesch_tag

    def detect_politeness(text):
        politeness_score = 0

        # Polite phrases
        polite_phrases = ["please", "kindly", "thank you", "would you", "could you", "I would appreciate", "sorry", "I apologize"]
        politeness_score += sum(phrase in text.lower() for phrase in polite_phrases)

        # Modal verbs
        modal_verbs = ["would", "could", "may", "might", "should"]
        politeness_score += sum(modal in text.lower() for modal in modal_verbs)

        # Indirect requests
        indirect_phrases = ["I was wondering", "is it possible", "could you please", "would it be okay if"]
        politeness_score += 2 * sum(phrase in text.lower() for phrase in indirect_phrases)

        # Greetings and closings
        greetings = ["dear", "hello", "good morning", "good evening"]
        closings = ["sincerely", "kind regards", "best regards", "yours faithfully"]
        politeness_score += sum(phrase in text.lower() for phrase in greetings + closings)

        # Gratitude and apologies
        gratitude_phrases = ["thank you", "I appreciate", "thanks", "I am grateful"]
        apology_phrases = ["I apologize", "sorry", "excuse me"]
        politeness_score += sum(phrase in text.lower() for phrase in gratitude_phrases + apology_phrases)

        # Hedging language
        hedging_phrases = ["it seems", "I think", "perhaps", "it might be better to"]
        politeness_score += sum(phrase in text.lower() for phrase in hedging_phrases)

        # Avoidance of commands
        if text.lower().startswith("send") or "do this" in text.lower():
            politeness_score -= 1

        # Sentence structure
        if len(text.split(',')) > 1:
            politeness_score += 1

        # Sentiment analysis
        from textblob import TextBlob
        sentiment = TextBlob(text).sentiment.polarity
        if sentiment > 0:
            politeness_score += 1

        # Final classification
        if politeness_score > 6:
            return politeness_score,"Highly Polite"
        elif 3 < politeness_score <= 6:
            return politeness_score,"Moderately Polite"
        else:
            return politeness_score,"Impolite or Neutral"

    def coherence(content):
        sentences=content.split(".")
        # Compute embeddings
        embeddings = coherency_analyzer.encode(sentences, convert_to_tensor=True)

        # Compute pairwise cosine similarities
        similarities = util.pytorch_cos_sim(embeddings, embeddings)

        # Display coherence scores
        score=0
        for i in range(len(similarities)):
            for j in range(len(similarities[0])):
                if j!=i:
                    score+=similarities[i][j]
        n=len(sentences)
        coherence_score=score/((n**2)-n)
        coherence_tag = "Excellent" if coherence_score >=0.35 else "Good" if 0.20< coherence_score <0.35 else "Poor"

        return round(float(coherence_score),2),coherence_tag

    def email_structure(email):
        structure_categories={
            "subject":["Subject"],

            "greeting":[
                      "Dear", "Hi", "Hello", "To Whom It May Concern", "Respected",
                      "Good Morning", "Good Afternoon", "Good Evening", "Greetings",
                      "Dear Sir", "Dear Madam","Dear Team", "Dear All", "Hi Everyone", "Hello Team",
                      "Dear Colleagues", "Dear Members"
                      ],

            "closing":[
                      "Best regards", "Sincerely", "Warm regards", "Thank you", "Thanking you", "Thanks",
                      "Yours sincerely", "Yours faithfully", "Kind regards", "Warm wishes", "Best wishes",
                      "Regards", "Thanks and regards", "Cheers", "Take care", "Respectfully",
                      "Looking forward to your response", "With appreciation", "Cordially",
                      "Yours truly", "Faithfully yours"
                     ]
        }

        lines = email.strip().split("\n")

        results = {
            "has_subject": False,
            "has_greeting": False,
            "has_closing": False,
        }

        score=0

        #check for subject
        for line in lines:
            if line.startswith("Subject"):
                score=score+1 if results["has_subject"] == False else score
                results["has_subject"] = True
                break

        #check for greeting
        for line in lines:
            for greeting in structure_categories["greeting"]:
                if greeting in line:
                    score=score+1 if results["has_greeting"] == False else score
                    results["has_greeting"] = True
                    break

        #check for closing
        for line in lines:
            for closing in structure_categories["closing"]:
                if closing in line:
                    score=score+1 if results["has_closing"] == False else score
                    results["has_closing"] = True
                    break


        return score,results


    #score
    score=0

    # Perform Assessments
    spell_inc_score, corrected_content = evaluate_spell_with_textblob(content)
    grammatical_inc_score, grammatical_score_of_10, grammatical_errors = grammatical_error(content)
    score+=grammatical_score_of_10
    relevance_score, relevancy_tag = evaluate_relevance(str(corrected_content), topic)
    score=score+10 if relevancy_tag=="Highly Relevant" else score+9 if relevancy_tag=="Relevant" else score+8
    sentiment_score, sentiment_tag = evaluate_sentiment(str(corrected_content))
    score=score+10 if sentiment_tag=="Positive" else score+9
    readability_score, readability_tag = evaluate_readability(str(corrected_content))
    score=score+10 if readability_tag=="Easy to Read" else score+9 if readability_tag=="Moderate Readability" else score+8
    politeness_score, politeness_tag = detect_politeness(str(corrected_content))
    score=score+10 if politeness_tag=="Highly Polite" else score+9 if politeness_tag=="Moderately Polite" else score+8
    coherency_score, coherency_tag = coherence(str(corrected_content))
    score=score+10 if coherency_tag=="Excellent" else score+9 if coherency_tag=="Good" else score+8
    email_structure_score, email_structure_metric = email_structure(content)
    score+=email_structure_score

    #normalizing score
    score=round((score*10)/63,2)


    # Generate Feedback
    feedback = {
        "Grammar incorections": grammatical_inc_score,
        "Grammar_score": f"{grammatical_score_of_10}/10",
        "Grammatical_errors": grammatical_errors,
        "Relevancy_type": relevancy_tag,
        "Sentiment_type": sentiment_tag,
        "Readability_type": readability_tag,
        "Politeness type": politeness_tag,
        "Coherency_type": coherency_tag,
        "Email_structure_score": f"{email_structure_score}/3",
        "Email_structure_type": email_structure_metric,
        "Overall Score": f"{score}/10"
    }

    # Display Feedback
    for key, value in feedback.items():
        print(f"{key}: {value}")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


[Gramformer] Grammar error correct/highlight model loaded..


In [14]:
#Grammar Errors: A high count indicates issues with clarity and professionalism.
#Relevance Score: A score close to 1.0 means the content is highly relevant to the topic.
#Sentiment Score: Positive scores indicate engaging and optimistic tones, while negative scores suggest the opposite.
#Readability Score: Higher scores (60-100) are easier to read for a general audience.

In [17]:
user_content="""
Subject: Request for Extension on Project Deadline

Dear Abhay,

I hope this email finds you well. I am writing to request a possible extension for the submission of the deadline, which is currently set for.

Due to unforeseen circumstances, our team has encountered delays in. We believe that an extension of would allow us to deliver work of higher quality that meets the expectations set for this project.

We deeply appreciate your understanding and are willing to discuss this matter further if needed. Please let us know if a formal meeting is required to address this request.

Thank you for considering our situation. We value your support and guidance.

Best regards,
Rakesh

"""
topic = "Deadline Extension"

presentation_skill_ass(user_content,topic)

Grammar incorections: 2
Grammar_score: 8.0/10
Grammatical_errors: {'Subject Request for Extension on Project Deadline': 'Subject Request for Extension of Project Deadline', 'We believe that an extension of would allow us to deliver work of higher quality that meets the expectations set for this project': 'We believe that an extension would allow us to deliver work of higher quality that meets the expectations set for this project'}
Relevancy_type: Highly Relevant
Sentiment_type: Positive
Readability_type: Moderate Readability
Politeness type: Moderately Polite
Coherency_type: Good
Email_structure_score: 3/3
Email_structure_type: {'has_subject': True, 'has_greeting': True, 'has_closing': True}
Overall Score: 9.206349206349206
